In [38]:
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
import UtilNNDL as util

from keras.models import Sequential
from keras.layers import Dense, Permute
from keras.layers import ConvLSTM2D, Conv2D, MaxPooling2D
from keras.layers import MaxPooling1D, Conv1D
from keras.layers import GRU, LSTM, BatchNormalization
from keras.layers import TimeDistributed
from keras.layers import Flatten, Reshape
from keras.utils import to_categorical
from keras.layers import Activation

A01T = h5py.File('datasets/A01T_slice.mat','r')
data = np.copy(A01T['image'])
labels = np.copy(A01T['type'])
labels = labels[0,0:data.shape[0]:1]
labels = np.asarray(labels, dtype=np.int32)

a = data[:56]
b = data[57:]
data = np.vstack((a,b))
data = data[:,:22,:]
a = labels[:56]
b = labels[57:]
labels = np.hstack((a,b))
print data.shape

(287, 22, 1000)


In [39]:
data2d_sliced, labels_sliced = util.create_window_data(data, labels)
#util.plot_hist(data2d_sliced)
print data2d_sliced.shape, labels_sliced.shape
data2d = data2d_sliced.reshape(data2d_sliced.shape[0], data2d_sliced.shape[2], data2d_sliced.shape[1], 1)
print data2d.shape

#enc = OneHotEncoder()
#enc_labels = enc.fit_transform(labels.reshape(-1,1)).toarray()
enc_labels = to_categorical(labels_sliced-769, num_classes=4)
print(enc_labels.shape)

(2870, 22, 512) (2870,)
(2870, 512, 22, 1)
(2870, 4)


In [51]:
bs, t, n, c = data2d.shape
np.random.seed(42)
shuffle = np.random.choice(bs,bs,replace=False)

#mask = np.ones_like(data)
#mask[[:,:,21:24]] = True
#newdata = data[mask]
#np.delete(mask, :,:,24)
train_samples = bs - 50
train_data = data2d[shuffle[:train_samples],:,:]
train_labels = enc_labels[shuffle[:train_samples]]
test_data = data2d[shuffle[train_samples:],:,:]
test_labels = enc_labels[shuffle[train_samples:]]
print data2d.shape
print train_data.shape
print test_data.shape

(2870, 512, 22, 1)
(2820, 512, 22, 1)
(50, 512, 22, 1)


# TESTING


In [52]:
num_filters = 15
model = Sequential()
model.add(Conv2D(num_filters, kernel_size=(num_filters,1), activation='relu', data_format='channels_last', input_shape=(data2d.shape[1], data2d.shape[2], data2d.shape[3])))  
model.add(Conv2D(num_filters, kernel_size=(15,num_filters), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Permute((1,3,2)))

model.add(Conv2D(num_filters*2, kernel_size=(10,num_filters), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Permute((1,3,2)))

model.add(Conv2D(num_filters*4, kernel_size=(10,num_filters*2), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Permute((1,3,2)))

model.add(Conv2D(num_filters*8, kernel_size=(10,num_filters*4), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Reshape((num_filters*8,1)))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(16))
#model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(4, activation='softmax'))
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])
model.summary()
hist = model.fit(train_data,train_labels,epochs=10,validation_split=0.25,batch_size=32)
#model.test_on_batch(test_data, test_labels)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_136 (Conv2D)          (None, 498, 22, 15)       240       
_________________________________________________________________
conv2d_137 (Conv2D)          (None, 484, 8, 15)        50640     
_________________________________________________________________
max_pooling2d_109 (MaxPoolin (None, 161, 8, 15)        0         
_________________________________________________________________
permute_82 (Permute)         (None, 161, 15, 8)        0         
_________________________________________________________________
conv2d_138 (Conv2D)          (None, 152, 1, 30)        36030     
_________________________________________________________________
max_pooling2d_110 (MaxPoolin (None, 50, 1, 30)         0         
_________________________________________________________________
permute_83 (Permute)         (None, 50, 30, 1)         0         
__________

In [26]:
model = Sequential()
model.add(Conv2D(15, kernel_size=(10,1), activation='relu', data_format='channels_last', input_shape=(data2d.shape[1], data2d.shape[2], data2d.shape[3])))  
model.add(Conv2D(15, kernel_size=(15,10), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Permute((1,3,2)))

model.add(Conv2D(30, kernel_size=(10,15), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Permute((1,3,2)))

model.add(Conv2D(60, kernel_size=(10,30), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Permute((1,3,2)))

model.add(Conv2D(120, kernel_size=(10,60), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(3,1)))
model.add(Reshape((120,1)))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(16))
#model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(4, activation='softmax'))
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])
model.summary()
hist = model.fit(train_data,train_labels,epochs=10,validation_split=0.25,batch_size=32)
#model.test_on_batch(test_data, test_labels)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_91 (Conv2D)           (None, 503, 25, 15)       165       
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 489, 16, 15)       33765     
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 163, 16, 15)       0         
_________________________________________________________________
permute_55 (Permute)         (None, 163, 15, 16)       0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 154, 1, 30)        72030     
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 51, 1, 30)         0         
_________________________________________________________________
permute_56 (Permute)         (None, 51, 30, 1)         0         
__________